In [15]:
import requests
import pandas as pd
from urllib.request import Request, urlopen
from sodapy import Socrata
import json
import re
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import ipywidgets as wd
from ipywidgets import interact
import seaborn as sns
#import ptitprince as pt
import base64

In [16]:
with open("newSecret.json", "r") as f:
    config = json.load(f)

token = config["TOKEN_API_OD"]
user=config["user"]
passwd=config["pw"]
client_id=config["clientId"]
client_secret=config["clientSecret"]

## MAIN ACCESS FUNCTION

In [17]:
def get_access_token(client_id, client_secret):
    # URL para obtener el token de acceso
    auth_url = 'https://accounts.spotify.com/api/token'
    
    # Creación del encabezado de autorización con client_id y client_secret codificados en base64
    auth_header = {
        'Authorization': f'Basic {base64.b64encode((client_id + ":" + client_secret).encode()).decode()}'
    }
    
    # Datos necesarios para la petición POST
    auth_data = {
        'grant_type': 'client_credentials',
        'client_id': client_id,
        'client_secret': client_secret
    }
    
    # Realizar la petición POST
    auth_response = requests.post(auth_url, data=auth_data, headers=auth_header)
    
    # Extraer la respuesta y obtener el token de acceso
    auth_response_data = auth_response.json()
    access_token = auth_response_data['access_token']
    
    # Devolver el token de acceso
    return access_token


## CALL ME BY YOUR NAME

In [51]:
def get_artists_by_name(query, access_token, limit=50):
    search_url = 'https://api.spotify.com/v1/search'
    
    headers = {
        'Authorization': f'Bearer {access_token}'
    }
    
    params = {
        'q': f"name:\"{query}\"",
        'type': 'artist',
        'limit': limit
    }
    
    response = requests.get(search_url, headers=headers, params=params)
    response_data = response.json()
    
    if 'artists' in response_data:
        return [artist['id'] for artist in response_data['artists']['items']]
    else:
        print("Error en la respuesta de la API:", response_data)
        return []

In [19]:
access_token = get_access_token(client_id, client_secret)

In [20]:
def extract_artists_to_dataframe(response_data):
    # Lista para almacenar datos de artistas
    artist_list = []

    # Iterar sobre cada artista en la respuesta
    for artist in response_data['artists']:
        # Extraer la información relevante
        artist_info = {
            'Name': artist['name'],
            'Genres': ', '.join(artist['genres']),
            'Popularity': artist['popularity'],
            'Spotify_URL': artist['external_urls']['spotify'],
            'Followers': artist['followers']['total'],
            'Image_URL': artist['images'][0]['url'] if artist['images'] else None, # Toma la primera imagen, si existe
            'Artist_ID': artist['id'],
            'Artist_Type': artist['type'],
            'Artist_URI': artist['uri']
        }
        # Agregar a la lista
        artist_list.append(artist_info)

    # Convertir lista en DataFrame
    df_artists = pd.DataFrame(artist_list)

    return df_artists

In [7]:
def get_artists_from_spotify(token, artist_ids):
    """
    Fetch artist details from Spotify's API.
    
    :param access_token: Token for API authorization.
    :param artist_ids: List of artist IDs.
    :return: Response data as a dictionary.
    """
    
    # Endpoint de la API
    base_url = "https://api.spotify.com/v1/artists"
    
    # Headers de la petición (incluye el token de acceso para autorización)
    headers = {
        "Authorization": f"Bearer {token}"
    }
    
    # Parámetros de la petición (IDs de los artistas)
    params = {
        "ids": ",".join(artist_ids)
    }
    
    # Realizar la petición GET a la API de Spotify
    response = requests.get(base_url, headers=headers, params=params)
    
    # Convertir la respuesta en formato JSON a un diccionario de Python
    response_data = response.json()

    return response_data

""" Ejemplo de uso:

artist_ids = ["2CIMQHirSU0MQqyYHq0eOx", "57dN52uHvrHOxijzpIgu3E"]  # Reemplaza con los IDs de los artistas que quieras consultar
token=access_token
 Obtener respuesta de la API
response_data = get_artists_from_spotify(token, artist_ids)

# Procesar la respuesta con la función extract_artists_to_dataframe
df_artist = extract_artists_to_dataframe(response_data)
#print(df)
df_artist"""

' Ejemplo de uso:\n\nartist_ids = ["2CIMQHirSU0MQqyYHq0eOx", "57dN52uHvrHOxijzpIgu3E"]  # Reemplaza con los IDs de los artistas que quieras consultar\ntoken=access_token\n Obtener respuesta de la API\nresponse_data = get_artists_from_spotify(token, artist_ids)\n\n# Procesar la respuesta con la función extract_artists_to_dataframe\ndf_artist = extract_artists_to_dataframe(response_data)\n#print(df)\ndf_artist'

## THE INPUT

In [21]:
Spotify_List_Id='7M36AGwbD9OpGUZN3IWx7V'
#7M36AGwbD9OpGUZN3IWx7V

In [23]:
def get_band_from_playlist_id(token, playlist_id):
    """
    Fetch band names from a Spotify playlist.
    
    :param token: Token for API authorization.
    :param playlist_id: Spotify Playlist ID.
    :return: List of band names.
    """
    playlist_url = f"https://api.spotify.com/v1/playlists/{playlist_id}/tracks"
    headers = {"Authorization": f"Bearer {token}"}
    response = requests.get(playlist_url, headers=headers)
    
    if response.status_code == 200:
        playlist_data = response.json()
        bands = []
        for item in playlist_data['items']:
            # Cada 'item' contiene información de una pista, incluyendo sus artistas
            track_artists = item['track']['artists']
            for artist in track_artists:
                bands.append(artist['name'])
        return bands
    else:
        print(f"Error fetching playlist data: {response.status_code}")
        return []

# Uso de la función
# token = "tu_token_de_acceso"
# playlist_id = "id_de_tu_playlist"
# band_names = get_band_from_playlist_id(token, playlist_id)
# print(band_names)


In [24]:
band_names = get_band_from_playlist_id(access_token, Spotify_List_Id)

In [25]:
Band_Names=pd.DataFrame(band_names,
columns=['Band_Name'])
Band_Names

,Band_Name
0,Deformation
1,SYRACUSÆ
2,Jose Macario
3,Under Threat
4,Demonsquest
...,...
102,Epiphany.
103,Licantropia
104,ubergehen
105,Cosmology X


In [26]:
Band_Names.columns

Index(['Band_Name'], dtype='object')

In [13]:
token=access_token
for i in range(0,len(band_names)):
    names = get_artists_by_name(band_names[i], access_token)
    response_data_names = get_artists_from_spotify(token, names)
    df_metal_artist_names = extract_artists_to_dataframe(response_data_names)

KeyboardInterrupt: 

In [57]:
test_name=band_names[1]
named_test = get_artists_by_name(test_name, access_token)
named_test

['65uP1jjzWh4VISZCA4s3X7',
 '3tPNUZKUf9Pa8R84yWpSvO',
 '64iLtTMfnTAuuMbRk66boJ',
 '3s5NSrjy5sXASsWsm55iPy',
 '2tJE40GixcEoh9zwoBD07H',
 '4LOPGgD3DBWffxYypHDzry',
 '4lqBkVmOyYfnDQCm5mA5ko',
 '6xinS4Wgb422R7KrrJ9UYU',
 '5AB3ONEZzBBSvjYYStYgap',
 '6dX29Av0TojL5ZMOOgLliI',
 '5JDM5S7eyY3ypNOyS5e0ML',
 '6WjPUWcR55uboKEjvH1Pt7',
 '0axxjRxJBx2ciiPYRcBPJ2',
 '4JmS6FBtKxZ9sQwS4Ezk4j',
 '59mntshH5MOPANbk9f69hP',
 '4ZiIj8ThbKaep4Kc0V5xbZ',
 '3ntNlXR1aenb9hNEL02WYH',
 '245qp5aDyBGuhSFu8GGSJi',
 '0I4flbOXVYUts2wY3cPt1R',
 '4TSO4eUt8kze7Ac5Gg1y8e',
 '2LXgEluAGEdhhBqTYtve0g',
 '2rD1kXQRKt6KpmKnLAZZ2E',
 '08cAZWaoVZnCrDCDY6P7tk',
 '0KIxr4HfjzUEBKy6vlfR9O',
 '7sUmD0wgjQR7cavOCCdP5W',
 '2JTmMIiuyMwoO9nvPoZJ2Z',
 '4XZ4x9dUVCLnrm8oRQMlDt',
 '1qo7mKjITlrCOs9nptnPEb',
 '0hKhw4tF73tq6390kUouxM']

In [58]:
token=access_token
response_data_names_test = get_artists_from_spotify(token, named_test)
df_metal_artist_named_test = extract_artists_to_dataframe(response_data_names_test)
Name_test=band_names[1]
locked_test = df_metal_artist_named_test.loc[df_metal_artist_named_test['Name'] == Name_test, 'Artist_ID']
locked_test

3    3s5NSrjy5sXASsWsm55iPy
Name: Artist_ID, dtype: object

In [53]:
named = get_artists_by_name('SYRACUSÆ', access_token)
named

['65uP1jjzWh4VISZCA4s3X7',
 '3tPNUZKUf9Pa8R84yWpSvO',
 '64iLtTMfnTAuuMbRk66boJ',
 '3s5NSrjy5sXASsWsm55iPy',
 '2tJE40GixcEoh9zwoBD07H',
 '4LOPGgD3DBWffxYypHDzry',
 '4lqBkVmOyYfnDQCm5mA5ko',
 '6xinS4Wgb422R7KrrJ9UYU',
 '5AB3ONEZzBBSvjYYStYgap',
 '6dX29Av0TojL5ZMOOgLliI',
 '5JDM5S7eyY3ypNOyS5e0ML',
 '6WjPUWcR55uboKEjvH1Pt7',
 '0axxjRxJBx2ciiPYRcBPJ2',
 '4JmS6FBtKxZ9sQwS4Ezk4j',
 '59mntshH5MOPANbk9f69hP',
 '4ZiIj8ThbKaep4Kc0V5xbZ',
 '3ntNlXR1aenb9hNEL02WYH',
 '245qp5aDyBGuhSFu8GGSJi',
 '0I4flbOXVYUts2wY3cPt1R',
 '4TSO4eUt8kze7Ac5Gg1y8e',
 '2LXgEluAGEdhhBqTYtve0g',
 '2rD1kXQRKt6KpmKnLAZZ2E',
 '08cAZWaoVZnCrDCDY6P7tk',
 '0KIxr4HfjzUEBKy6vlfR9O',
 '7sUmD0wgjQR7cavOCCdP5W',
 '2JTmMIiuyMwoO9nvPoZJ2Z',
 '4XZ4x9dUVCLnrm8oRQMlDt',
 '1qo7mKjITlrCOs9nptnPEb',
 '0hKhw4tF73tq6390kUouxM']

In [54]:
token=access_token
response_data_names = get_artists_from_spotify(token, named)
df_metal_artist_named = extract_artists_to_dataframe(response_data_names)
Name=band_names[1]
locked = df_metal_artist_named.loc[df_metal_artist_named['Name'] == Name, 'Artist_ID']
locked

3    3s5NSrjy5sXASsWsm55iPy
Name: Artist_ID, dtype: object

In [50]:
df_metal_artist_named

,Name,Genres,Popularity,Spotify_URL,Followers,Image_URL,Artist_ID,Artist_Type,Artist_URI
0,Paid Master Received Contract Signed Editor St...,,0,https://open.spotify.com/artist/65uP1jjzWh4VIS...,46,None,65uP1jjzWh4VISZCA4s3X7,artist,spotify:artist:65uP1jjzWh4VISZCA4s3X7
1,Syracuse Crunch,,5,https://open.spotify.com/artist/3tPNUZKUf9Pa8R...,35,https://i.scdn.co/image/ab6761610000e5ebe18fb5...,3tPNUZKUf9Pa8R84yWpSvO,artist,spotify:artist:3tPNUZKUf9Pa8R84yWpSvO
2,10NamesJune,,0,https://open.spotify.com/artist/2tJE40GixcEoh9...,28,https://i.scdn.co/image/ab6761610000e5eb8a2fac...,2tJE40GixcEoh9zwoBD07H,artist,spotify:artist:2tJE40GixcEoh9zwoBD07H
3,Syracuse University Singers,,7,https://open.spotify.com/artist/4LOPGgD3DBWffx...,38,https://i.scdn.co/image/ab67616d0000b273eab73a...,4LOPGgD3DBWffxYypHDzry,artist,spotify:artist:4LOPGgD3DBWffxYypHDzry
4,NAME101,,0,https://open.spotify.com/artist/4lqBkVmOyYfnDQ...,0,None,4lqBkVmOyYfnDQCm5mA5ko,artist,spotify:artist:4lqBkVmOyYfnDQCm5mA5ko
5,Syracuse University,college marching band,2,https://open.spotify.com/artist/6xinS4Wgb422R7...,55,None,6xinS4Wgb422R7KrrJ9UYU,artist,spotify:artist:6xinS4Wgb422R7KrrJ9UYU
6,1000names,,3,https://open.spotify.com/artist/64iLtTMfnTAuuM...,1967,https://i.scdn.co/image/ab67616d0000b2738e3ba9...,64iLtTMfnTAuuMbRk66boJ,artist,spotify:artist:64iLtTMfnTAuuMbRk66boJ
7,Syriac Music Choir,syro-aramaic chant,13,https://open.spotify.com/artist/6WjPUWcR55uboK...,898,https://i.scdn.co/image/ab67616d0000b2732688b4...,6WjPUWcR55uboKEjvH1Pt7,artist,spotify:artist:6WjPUWcR55uboKEjvH1Pt7
8,J. Money 1st Name Last Name (mr. Futuristic),futuristic swag,0,https://open.spotify.com/artist/59mntshH5MOPAN...,1,None,59mntshH5MOPANbk9f69hP,artist,spotify:artist:59mntshH5MOPANbk9f69hP
9,Sirarcusa,,6,https://open.spotify.com/artist/4JmS6FBtKxZ9sQ...,87,None,4JmS6FBtKxZ9sQwS4Ezk4j,artist,spotify:artist:4JmS6FBtKxZ9sQwS4Ezk4j


In [49]:
Name = band_names[1]
print("Buscando:", Name)  # Esto te mostrará qué estás buscando exactamente

# Verifica si el nombre existe en el DataFrame
print(Name in df_metal_artist_named['Name'].values)

# Intenta el filtrado
locked = df_metal_artist_named.loc[df_metal_artist_named['Name'] == Name, 'Artist_ID']
print(locked)


Buscando: SYRACUSÆ
False
Series([], Name: Artist_ID, dtype: object)


In [32]:
df_metal_artist_named

,Name,Genres,Popularity,Spotify_URL,Followers,Image_URL,Artist_ID,Artist_Type,Artist_URI
0,Paid Master Received Contract Signed Editor St...,,0,https://open.spotify.com/artist/65uP1jjzWh4VIS...,46,None,65uP1jjzWh4VISZCA4s3X7,artist,spotify:artist:65uP1jjzWh4VISZCA4s3X7
1,10NamesJune,,0,https://open.spotify.com/artist/2tJE40GixcEoh9...,28,https://i.scdn.co/image/ab6761610000e5eb8a2fac...,2tJE40GixcEoh9zwoBD07H,artist,spotify:artist:2tJE40GixcEoh9zwoBD07H
2,NAME101,,0,https://open.spotify.com/artist/4lqBkVmOyYfnDQ...,0,None,4lqBkVmOyYfnDQCm5mA5ko,artist,spotify:artist:4lqBkVmOyYfnDQCm5mA5ko
3,1000names,,3,https://open.spotify.com/artist/64iLtTMfnTAuuM...,1967,https://i.scdn.co/image/ab67616d0000b2738e3ba9...,64iLtTMfnTAuuMbRk66boJ,artist,spotify:artist:64iLtTMfnTAuuMbRk66boJ
4,J. Money 1st Name Last Name (mr. Futuristic),futuristic swag,0,https://open.spotify.com/artist/59mntshH5MOPAN...,1,None,59mntshH5MOPANbk9f69hP,artist,spotify:artist:59mntshH5MOPANbk9f69hP
5,119 No Names,,0,https://open.spotify.com/artist/5AB3ONEZzBBSvj...,43,https://i.scdn.co/image/ab6761610000e5eb3a6568...,5AB3ONEZzBBSvjYYStYgap,artist,spotify:artist:5AB3ONEZzBBSvjYYStYgap
6,NAMELESS1,,0,https://open.spotify.com/artist/0axxjRxJBx2cii...,1,https://i.scdn.co/image/ab67616d0000b273ea797e...,0axxjRxJBx2ciiPYRcBPJ2,artist,spotify:artist:0axxjRxJBx2ciiPYRcBPJ2
7,Namesake 106:8,,0,https://open.spotify.com/artist/0I4flbOXVYUts2...,1,https://i.scdn.co/image/ab67616d0000b273c24f70...,0I4flbOXVYUts2wY3cPt1R,artist,spotify:artist:0I4flbOXVYUts2wY3cPt1R
8,1000names,,0,https://open.spotify.com/artist/5JDM5S7eyY3ypN...,0,None,5JDM5S7eyY3ypNOyS5e0ML,artist,spotify:artist:5JDM5S7eyY3ypNOyS5e0ML
9,2Rons1Name,,0,https://open.spotify.com/artist/2LXgEluAGEdhhB...,0,None,2LXgEluAGEdhhBqTYtve0g,artist,spotify:artist:2LXgEluAGEdhhBqTYtve0g


In [ ]:
for i in range(0, len(BMR_Tracks_List), batch_size):
    batch_ids = BMR_Tracks_List[i:i+batch_size]
    batch_data = get_features_from_tracks(token, batch_ids, batch_size)
    df_batch = extract_features_to_dataframe(batch_data)  # Asumiendo que esta función ya está definida
    df_BMR_Features = pd.concat([df_BMR_Features, df_batch], ignore_index=True)